In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [36]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v0.6", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who generates a brief counter-arguments to any statement",
    },
    {"role": "user", "content": "Vaccine are bad for health "},
]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
# print(outputs[0]["generated_text"])


Device set to use cuda:0


In [37]:
# LABEL DESCRIPTIONS

labels_map = {}
labels_map['religious'] = "religious beliefs and their influence on views about vaccines"
labels_map['political'] = "the political factors that affect perceptions of vaccine use"
labels_map['ingredients'] = "concerns about the ingredients and chemical components in vaccines"
labels_map['unnecessary'] = "the importance and necessity of getting vaccinated to prevent diseases"
labels_map['conspiracy'] = "conspiracy theories suggesting hidden motives behind vaccination efforts"
labels_map['mandatory'] = "the debate over personal choice versus mandates in vaccination policies"
labels_map['ineffective'] = "evidence and reasons that support the effectiveness of vaccines"
labels_map['side-effect'] = "potential side effects and adverse reactions associated with vaccines"
labels_map['pharma'] = "the role of pharmaceutical companies and concerns about profit motives"
labels_map['rushed'] = "claims that vaccines were approved or developed without sufficient testing"
labels_map['country'] = "national biases and objections to vaccines produced by specific countries"

In [38]:
def create_message_column_with_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    # print(row)
    prompt= f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n Talk About "

    lab=row['labels'].split()
    if isinstance(lab, list):
            mapped_labels = " and ".join([labels_map.get(l, "") for l in lab])
    else:
            mapped_labels = labels_map.get(lab, "")
    # print(mapped_labels)
    prompt+= mapped_labels
    prompt+= " ##Output: "
    user = {
        "content": prompt,
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


In [39]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_with_label_desc.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_data_with_labels_desc.csv',split='train')

# train_data=train_data[:5]
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 5244.58 examples/s]


In [11]:
len(train_data), len(val_data)

(2000, 987)

In [40]:
def create_message_column_without_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
#     print(row['text'])
    user = {
        "content": f"Generate Counter Argument for the tweet:\n Tweet: {row['text']}",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [26]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_without_labels.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_CA_without_labels.csv',split='train')

In [8]:
dataset_chatml = train_data.map(create_message_column_with_labels)

dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map: 100%|██████████| 1500/1500 [00:00<00:00, 6238.11 examples/s]


In [27]:
tokenizer=pipe.tokenizer

In [41]:
import torch
import peft ,trl

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
from trl import SFTTrainer

In [9]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v0.6"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [42]:

if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# # This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [43]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token if none is defined

# Set the pad_token_id
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# Set padding side if needed
tokenizer.padding_side = 'right'
# set cuda available device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

use_4bit = True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True
bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto",
#           attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model = prepare_model_for_kbit_training(model)
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
peft_config = LoraConfig(
        r=4,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)
# model = get_peft_model(model, peft_config)


Using GPU: NVIDIA RTX A5000


In [13]:
args = TrainingArguments(
        output_dir="//home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/tiny-llama-1.1b-chat-v0.6",
        evaluation_strategy="epoch",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        learning_rate=1e-4,
        logging_dir=None,  # Directory for storing logs
        logging_steps=100,     # Log after every 100 steps
        logging_strategy="steps",  # Log by steps instead of epochs
        fp16=True,
#         bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=2,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
#     report_to="wandb",
        seed=42,
)

/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
for sample in dataset_chatml.select(range(5)):
    print(sample["text"])

<|user|>
Generate Counter Argument for the tweet:
 Tweet: #CORRUPTION by pushing through #Pfizer vaccine which didn ’ t do a peer review & were able to push to the finish line & why ? Because , they are #BigPhama 2 many ? About the data ! @InovioPharma is transparent & submitted a peer review over 4 months ago & still no answer ? The news</s>
<|assistant|>
While it is important to have transparency and peer review in the development of vaccines, it is also crucial to prioritize speed in a situation like a global pandemic. Pfizer may have pushed through their vaccine quickly because the need for a vaccine was urgent. Additionally, Pfizer has a long history of successful vaccine development and regulatory approval. Just because Inovio Pharma submitted a peer review earlier does not necessarily mean their vaccine is a better option. Fast-tracking the Pfizer vaccine could have been a necessary step in quickly providing protection to the public.</s>

<|user|>
Generate Counter Argument for t

[codecarbon INFO @ 11:33:53] Energy consumed for RAM : 0.006271 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:33:53] Energy consumed for all GPUs : 0.012726 kWh. Total GPU Power : 73.92578068094876 W
[codecarbon INFO @ 11:33:53] Energy consumed for all CPUs : 0.004154 kWh. Total CPU Power : 28.75423890701124 W
[codecarbon INFO @ 11:33:53] 0.023151 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:33:55] Energy consumed for RAM : 0.012934 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:33:55] Energy consumed for all GPUs : 0.030090 kWh. Total GPU Power : 73.77539265292617 W
[codecarbon INFO @ 11:33:55] Energy consumed for all CPUs : 0.009168 kWh. Total CPU Power : 28.765686692893773 W
[codecarbon INFO @ 11:33:55] 0.052192 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:33:55] Energy consumed for RAM : 0.029004 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:33:55] Energy consumed for all GPUs : 0.058549 kWh. Total

In [14]:
import os
from rouge_score import rouge_scorer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
print(torch.cuda.current_device())

# response_template = "</s>\n<|assistant|>\n"
# instruction_template = "<s> <|user|>"
# collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,response_template=response_template, tokenizer=tokenizer)
        # Round the values for better readability
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml,
        eval_dataset=val_dataset_chatml,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=1024,
#         compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        args=args,
        # data_collator=collator
)
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()


0


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:2065: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: Futur

/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.430400,1.411130
2,1.333700,1.390309


[codecarbon INFO @ 23:16:13] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 23:16:13] Energy consumed for all GPUs : 0.001010 kWh. Total GPU Power : 242.2229634121177 W
[codecarbon INFO @ 23:16:13] Energy consumed for all CPUs : 0.000228 kWh. Total CPU Power : 54.78933713291652 W
[codecarbon INFO @ 23:16:13] 0.001435 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:16:28] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 23:16:28] Energy consumed for all GPUs : 0.002061 kWh. Total GPU Power : 252.52244408628528 W
[codecarbon INFO @ 23:16:28] Energy consumed for all CPUs : 0.000455 kWh. Total CPU Power : 54.476291124273025 W
[codecarbon INFO @ 23:16:28] 0.002909 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:16:43] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 23:16:43] Energy consumed for all GPUs : 0.003128 kWh. Tota

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
model_path = "/home/sohampoddar/HDD2/utsav/Model_Codes_with_labels/tiny-llama-1.1b-chat-v0.6"
tokenizer_path = "/home/sohampoddar/HDD2/utsav/Model_Codes_with_labels/tiny-llama-1.1b-chat-v0.6"
# connect to device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Using GPU: NVIDIA RTX A5000


In [14]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

In [44]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
val_dataset_chatml=val_data.map(create_message_column_without_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)
dataset_chatml = val_dataset_chatml.
dataset_chatml

{'labels': ["['conspiracy', 'pharma']",
  "['ineffective']",
  "['none']",
  "['pharma', 'rushed']",
  "['political']",
  "['side-effect']",
  "['mandatory']",
  "['side-effect']",
  "['rushed']",
  "['pharma']",
  "['ineffective']",
  "['rushed']",
  "['side-effect']",
  "['conspiracy', 'ingredients']",
  "['conspiracy', 'mandatory']",
  "['rushed', 'side-effect']",
  "['unnecessary']",
  "['side-effect']",
  "['ineffective']",
  "['rushed']",
  "['none']",
  "['pharma']",
  "['side-effect']",
  "['side-effect']",
  "['side-effect']",
  "['mandatory']",
  "['ineffective']",
  "['mandatory', 'rushed']",
  "['rushed']",
  "['none']",
  "['political']",
  "['side-effect']",
  "['country']",
  "['political', 'rushed']",
  "['side-effect']",
  "['pharma', 'side-effect']",
  "['side-effect']",
  "['ineffective', 'side-effect']",
  "['ineffective']",
  "['none']",
  "['side-effect']",
  "['ineffective']",
  "['side-effect']",
  "['pharma']",
  "['mandatory']",
  "['rushed']",
  "['side-effec

In [52]:
len(dataset_chatml)

4

In [31]:
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2, max_time= 50)
    # print(outputs[0]['generated_text'])
    return outputs[0]['generated_text'][len(prompt):].strip()

In [55]:
labels = [dataset_chatml['labels'][i] for i in range(50)]
labels
# extract the tweets 
# tweets = [dataset_chatml['text'][i] for i in range(50)]
# labels, tweets

["['conspiracy', 'pharma']",
 "['ineffective']",
 "['none']",
 "['pharma', 'rushed']",
 "['political']",
 "['side-effect']",
 "['mandatory']",
 "['side-effect']",
 "['rushed']",
 "['pharma']",
 "['ineffective']",
 "['rushed']",
 "['side-effect']",
 "['conspiracy', 'ingredients']",
 "['conspiracy', 'mandatory']",
 "['rushed', 'side-effect']",
 "['unnecessary']",
 "['side-effect']",
 "['ineffective']",
 "['rushed']",
 "['none']",
 "['pharma']",
 "['side-effect']",
 "['side-effect']",
 "['side-effect']",
 "['mandatory']",
 "['ineffective']",
 "['mandatory', 'rushed']",
 "['rushed']",
 "['none']",
 "['political']",
 "['side-effect']",
 "['country']",
 "['political', 'rushed']",
 "['side-effect']",
 "['pharma', 'side-effect']",
 "['side-effect']",
 "['ineffective', 'side-effect']",
 "['ineffective']",
 "['none']",
 "['side-effect']",
 "['ineffective']",
 "['side-effect']",
 "['pharma']",
 "['mandatory']",
 "['rushed']",
 "['side-effect']",
 "['unnecessary']",
 "['side-effect']",
 "['side-ef

In [34]:
# extract the tweets and labels and store in a csv file 
import pandas as pd
import numpy as np
import os

# Load the dataset


0
side-effect
 Talk About potential side effects and adverse reactions associated with vaccines ##Output: </s>
1
ineffective political rushed
 Talk About evidence and reasons that support the effectiveness of vaccines and the political factors that affect perceptions of vaccine use and claims that vaccines were approved or developed without sufficient testing ##Output: </s>
2
side-effect
 Talk About potential side effects and adverse reactions associated with vaccines ##Output: </s>
3
mandatory
 Talk About the debate over personal choice versus mandates in vaccination policies ##Output: </s>
4
pharma
 Talk About the role of pharmaceutical companies and concerns about profit motives ##Output: </s>
5
mandatory
 Talk About the debate over personal choice versus mandates in vaccination policies ##Output: </s>
6
pharma side-effect
 Talk About the role of pharmaceutical companies and concerns about profit motives and potential side effects and adverse reactions associated with vaccines ##Out

[codecarbon INFO @ 11:52:51] Energy consumed for RAM : 0.021557 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:52:51] Energy consumed for all GPUs : 0.075207 kWh. Total GPU Power : 75.5788176498395 W
[codecarbon INFO @ 11:52:51] Energy consumed for all CPUs : 0.018413 kWh. Total CPU Power : 29.781814168079226 W
[codecarbon INFO @ 11:52:51] 0.115176 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:52:51] Energy consumed for RAM : 0.030376 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:52:51] Energy consumed for all GPUs : 0.096614 kWh. Total GPU Power : 75.61625204344524 W
[codecarbon INFO @ 11:52:51] Energy consumed for all CPUs : 0.024787 kWh. Total CPU Power : 29.784133225673457 W
[codecarbon INFO @ 11:52:51] 0.151777 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:52:53] Energy consumed for RAM : 0.021165 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 11:52:53] Energy consumed for all GPUs : 0.073635 kWh. Total

In [22]:
test_inference(dataset_chatml['test'][1]['messages'][0]['content'])

"While it's important to acknowledge concerns about the speed at which some COVID-19 vaccines, including those from Pfizer, were authorized for public use, it's equally crucial to consider the extensive research behind their development. The rigorous clinical trial process involved thousands of participants in diverse populations, with multiple phases evaluating efficacy and safety before approval. Regulatory agencies like the FDA and EMA conducted thorough reviews based on substantial data gathered"

In [20]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [21]:
num_samples=50
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=1, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2,
                   max_time= 180)
# print(outputs)
preds = []
for i in range(len(outputs)):
    generated_text = outputs[i][0]['generated_text']
  
    response = generated_text[len(prompts[i]):].split()
    # print(response)
    if len(response) > 1:
        # Extract the counter argument
        pred = " ".join(response)
    else:
        pred = ""  # Handle case with no valid split
    preds.append(pred)

    # Print prediction and corresponding reference
    if i < 51:
        print(f"Prediction {i }: {pred}")
        print(f"Reference {i}: {dataset_chatml['test'][i]['counter_argument']}")
        print("---")  # Separator for clarity
references= [dataset_chatml['test'][i]['counter_argument'] for i in range(len(outputs))]
metric.add_batch(predictions=preds, references=references)
tweets = [dataset_chatml['test'][i]['text'] for i in range(len(outputs))]

# store the tweets predictions and references in a csv file 
import pandas as pd
df = pd.DataFrame(list(zip(tweets, preds, references)), columns =['tweets', 'predictions', 'references'])
df.to_csv('/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/tiny_llama_cross_2_labels.csv', index=False)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


Prediction 0: While it is important to be skeptical of any new medical technology, it's essential to consider that millions have already been vaccinated against Covid-19 without any serious side effects or complications. The benefits of getting vaccinated far outweigh the risks and should not be dismissed as a conspiracy theory. It's also crucial to remember that the vaccines are continuously monitored by health authorities worldwide to ensure their
Reference 0: It's important to remember that vaccines undergo rigorous testing and approval processes before being distributed to the public. Trusting in the science behind the vaccine can help protect not only yourself, but also those around you.
---
Prediction 1: There is no legal loophole to avoid getting vaccinated, as it is mandatory by law. The COVID-19 virus does not have any immunity passports or exemptions that can be waived. It's important to trust in the scientific consensus and get vaccinated to protect yourself and others from 

In [22]:
metric.add_batch(predictions=preds, references=references)
result = metric.compute(use_stemmer=True)

In [23]:
import numpy as np

# Assuming result contains your ROUGE scores
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print("Rouge 1 Mean: ", rouge1_mean)
print("Rouge 2 Mean: ", rouge2_mean)
print("Rouge L Mean: ", rougeL_mean)
print("Rouge Lsum Mean: ", rougeLsum_mean)

Rouge 1 Mean:  36.253855498950706
Rouge 2 Mean:  9.763647326510581
Rouge L Mean:  22.706028592160642
Rouge Lsum Mean:  22.765535273227695


In [25]:
!pip install bert-score

    PyYAML (>=5.1.*)
            ~~~~~~^


In [24]:
from bert_score import score
P, R, F1 = score(preds, references, lang="en", verbose=True)

# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

loading configuration file config.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/sohampoddar/HDD2/hfcache/hub/models--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/vo

calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  3.92it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 54.64it/s]

done in 0.54 seconds, 92.76 sentences/sec
BERTScore Precision: 0.8702241778373718
BERTScore Recall: 0.8826459646224976
BERTScore F1: 0.8763450384140015
